Get all the pre requisites for the tf-idf vectorization and the knn model building 

In [33]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np 

In [34]:
df=pd.read_csv('PreProcessedData.csv')

In [35]:
df.head()

Unnamed: 0                                  name  \
0           0                   vaal ki usal recipe   
1           1          capsicum paneer sabzi recipe   
2           2  restaurant style palak paneer recipe   
3           3        beetroot and dill salad recipe   
4           4                  how to eat flaxseeds   

                                         ingredients  \
0  ['two', 'sprout', 'vaal', 'field', 'bean', 'bu...   
1  ['two', 'zero', 'point', 'five', 'capsicum', '...   
2  ['one', 'slice', 'onion', 'three', 'roughli', ...   
3  ['one', 'oliv', 'oil', 'one', 'vinegar', 'must...   
4  ['one', 'flax', 'seed', 'one', 'lemon', 'juic'...   

                                        instructions     time serving_size  \
0  For vaal ki usal To make vaal ki usal recipe, ...  34 Mins   4 servings   
1  For capsicum paneer sabziTo make capsicum pane...  26 Mins   4 servings   
2  For the onion-cashew paste Combine all the ing...  46 Mins   4 servings   
3  For beetroot and dill saladTo make beetroot an...  15 Mins   4 servings   
4  For flaxseeds mukhwas Combine all the ingredie...   5 Mins   4 servings   

                                                tags  calories  protein  \
0  ['nonstick', 'pan', 'boil', 'indian', 'recip',...     195.0     10.3   
1  ['non', 'stick', 'kadai', 'veg', 'antioxid', '...      74.0      2.6   
2  ['nonstick', 'pan', 'indian', 'dinner', 'india...     374.0     13.3   
3  ['refriger', 'indian', 'salad', 'indian', 'sal...      92.0      0.1   
4  ['indian', 'dinner', 'indian', 'veg', 'recip',...      68.0      2.3   

   carbohydrates  fiber   fat  sodium  
0           30.5    7.9   4.1     8.8  
1            9.5    3.2   2.9    21.8  
2           16.4    4.2  28.4    98.6  
3            0.4    0.0  10.0     0.0  
4            3.9    3.4   5.3   114.5

In [36]:
float_columns = ['calories', 'protein','carbohydrates','fiber','fat','sodium']  # Specify your float columns
for col in float_columns:
    df[col] = df[col].astype(str)  # Convert floats to strings
    # Optionally, handle NaN values before conversion
    df[col] = df[col].fillna('0')

In [37]:
df['combined']=df['ingredients']+" "+df['tags']+" "+df['calories']+" "+df['protein']+" "+df['carbohydrates']+df['fiber']+" "+df['fat']+" "+df['sodium']

In [38]:
df['combined'][0]

"['two', 'sprout', 'vaal', 'field', 'bean', 'butter', 'bean', 'one', 'oil', 'one', 'cumin', 'seed', 'jeera', 'asafoetida', 'hing', 'five', 'six', 'curri', 'leav', 'kadi', 'patta', 'zero', 'point', 'five', 'fine', 'chop', 'onion', 'two', 'gingergarl', 'adraklehsun', 'past', 'zero', 'point', 'two', 'five', 'fine', 'chop', 'tomato', 'zero', 'point', 'five', 'turmer', 'powder', 'haldi', 'two', 'malvani', 'masala', 'five', 'kokum', 'zero', 'point', 'five', 'chop', 'jaggeri', 'gur', 'salt', 'zero', 'point', 'two', 'five', 'fine', 'chop', 'coriand', 'dhania'] ['nonstick', 'pan', 'boil', 'indian', 'recip', 'saut', 'indian', 'veg', 'recip', 'maharashtrian', 'recip', 'maharashtrian', 'dal', 'varan', 'amti', 'kalvan', 'proteinrichfoodsforpregn', 'pregnancydalsandveget', 'lowcaloriedalkadhisforweightloss', 'ironrichvegetablesd', 'lowcholesterolsubzisd', 'diabeticdalkadhirecip', 'highproteind', 'calciumrichdalkadhi'] 195.0 10.3 30.57.9 4.1 8.8"

In [39]:
tfidf = TfidfVectorizer(
    stop_words='english',  # Remove common English stop words
    max_df=0.8,           # Ignore terms that appear in >80% of documents
    min_df=1,             # Adjust based on dataset size
    max_features=1000     # Limit to top 1000 features
)

In [40]:
tfidf_matrix = tfidf.fit_transform(df['combined'])

In [41]:
knn = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute')
knn.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [42]:
def get_recipe_recommendations(user_input, df, tfidf, knn, top_n=5):
    processed_input = user_input.lower().replace('[^a-z0-9\s]', '')
    processed_input = processed_input.replace(',', ' ')
    combined_input = processed_input
    input_tfidf = tfidf.transform([combined_input])
    distances, indices = knn.kneighbors(input_tfidf, n_neighbors=top_n)
    recommendations = [
        (df['name'].iloc[idx], distances[0][i])
        for i, idx in enumerate(indices[0])
    ]
    return recommendations

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Shahana\AppData\Local\Temp\ipykernel_12872\3785150610.py:2: SyntaxWarning: invalid escape sequence '\s'
  processed_input = user_input.lower().replace('[^a-z0-9\s]', '')


In [44]:
user_input = "tomato, basil, Italian with low calories"
recommendations = get_recipe_recommendations(user_input, df, tfidf, knn, top_n=5)
print(f"Recommendations for input '{user_input}':")
for i, (name, distance) in enumerate(recommendations):
    print(f"{i+1}. {name} ")

Recommendations for input 'tomato, basil, Italian with low calories':
1. creamy broccoli pasta recipe 
2. avocado and cherry tomato crostini recipe 
3. Low Calorie Canneloni ( Fast Foods Made Healthy Recipe) 
4. Tiramisu ( Italian Cookbook) 
5. Beef, feta and pesto meatballs 
